In [1]:
# imports and env't configs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import json
import requests 
from imp import reload 
import Util; reload(Util)
from datetime import datetime as dt
import matplotlib.dates as md
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# data reading 
data = pd.read_csv(Util.dataset, sep = ',', encoding = "ISO-8859-1")

In [3]:
# data prepareation 
data['dtime'] = data['dtime'].apply(Util.round_time_to_year)
data = data[(data['dtime'] >= '2014') & (data['dtime'] <= '2017')]

In [4]:
data['fetch_time'] = data['fetch_time'].div(1000)
data['ttfb_avg'] = data['ttfb_avg'].div(1000)
data['lookup_avg_time'] = data['lookup_avg_time'].div(1000)

In [5]:
data.count()

unit_id               7755073
dtime                 7755073
target                7755073
address               7754603
fetch_time            7755073
bytes_total           7755073
bytes_sec             7755073
objects               7755073
threads               7755073
requests              7755073
connections           7755073
reused_connections    7755073
lookups               7755073
request_total_time    7755073
request_min_time      7755073
request_avg_time      7755073
request_max_time      7755073
ttfb_total            7755073
ttfb_min              7755073
ttfb_avg              7755073
ttfb_max              7755073
lookup_total_time     7755073
lookup_min_time       7755073
lookup_avg_time       7755073
lookup_max_time       7755073
successes             7755073
failures              7755073
location_id           7755073
dtype: int64

In [6]:
data[['target', 'lookups', 'requests']].tail()

,target,lookups,requests
7982153,http://www.live.ca,0,0
7982154,http://www.cbc.ca,2,8
7982155,http://www.google.ca,1,4
7982156,http://www.ebay.com,9,65
7982157,http://ici.radio-canada.ca,10,81


In [7]:
(set(data['target'])) 

{'http://ca.yahoo.com',
 'http://ici.radio-canada.ca',
 'http://www-ofcom.samknows.com/static/mobile-test/',
 'http://www.cbc.ca',
 'http://www.ebay.com',
 'http://www.facebook.com/policies',
 'http://www.google.ca',
 'http://www.google.com/mobile/',
 'http://www.live.ca',
 'http://www.youtube.com'}

In [8]:
# Filter the dataset for each website ...
google = data[data['target'].str.contains("google.com")]
facebook = data[data['target'].str.contains("facebook.com")]
youtube = data[data['target'].str.contains("youtube.com")]

In [9]:
# Group data first by probe ID, then by dtime (time)
google_grouped = google.groupby(["unit_id", "dtime"])
facebook_grouped = facebook.groupby(["unit_id", "dtime"])
youtube_grouped = youtube.groupby(["unit_id", "dtime"])

In [10]:
# For each group, calculate the average of only the snow_depth column
google_ft_year_median = google_grouped.aggregate({"fetch_time":np.median})
google_ttfb_year_median = google_grouped.aggregate({"ttfb_avg":np.median})
google_dnst_year_median = google_grouped.aggregate({"lookup_avg_time":np.median})
google_objects_year_median = google_grouped.aggregate({"objects":np.median})
google_bytes_year_median = google_grouped.aggregate({"bytes_total":np.median})
google_lookups_year_median = google_grouped.aggregate({"lookups":np.median})

facebook_ft_year_median = facebook_grouped.aggregate({"fetch_time":np.median})
facebook_ttfb_year_median = facebook_grouped.aggregate({"ttfb_avg":np.median})
facebook_dnst_year_median = facebook_grouped.aggregate({"lookup_avg_time":np.median})
facebook_objects_year_median = facebook_grouped.aggregate({"objects":np.median})
facebook_bytes_year_median = facebook_grouped.aggregate({"bytes_total":np.median})
facebook_lookups_year_median = facebook_grouped.aggregate({"lookups":np.median})

youtube_ft_year_median = youtube_grouped.aggregate({"fetch_time":np.median})
youtube_ttfb_year_median = youtube_grouped.aggregate({"ttfb_avg":np.median})
youtube_dnst_year_median = youtube_grouped.aggregate({"lookup_avg_time":np.median})
youtube_objects_year_median = youtube_grouped.aggregate({"objects":np.median})
youtube_bytes_year_median = youtube_grouped.aggregate({"bytes_total":np.median})
youtube_lookups_year_median = youtube_grouped.aggregate({"lookups":np.median})

In [11]:
ggl_df = google_ttfb_year_median.reset_index(level=['unit_id', 'dtime'])
ggl_df['fetch_time'] = google_ft_year_median.reset_index(level=['unit_id', 'dtime'])['fetch_time']
ggl_df['lookup_avg_time'] = google_dnst_year_median.reset_index(level=['unit_id', 'dtime'])['lookup_avg_time']
ggl_df['objects'] = google_objects_year_median.reset_index(level=['unit_id', 'dtime'])['objects']
ggl_df['bytes_total'] = google_bytes_year_median.reset_index(level=['unit_id', 'dtime'])['bytes_total']
ggl_df['lookups'] = google_lookups_year_median.reset_index(level=['unit_id', 'dtime'])['lookups']

fb_df = facebook_ttfb_year_median.reset_index(level=['unit_id', 'dtime'])
fb_df['fetch_time'] = facebook_ft_year_median.reset_index(level=['unit_id', 'dtime'])['fetch_time']
fb_df['lookup_avg_time'] = facebook_dnst_year_median.reset_index(level=['unit_id', 'dtime'])['lookup_avg_time']
fb_df['objects'] = facebook_objects_year_median.reset_index(level=['unit_id', 'dtime'])['objects']
fb_df['bytes_total'] = facebook_bytes_year_median.reset_index(level=['unit_id', 'dtime'])['bytes_total']
fb_df['lookups'] = facebook_lookups_year_median.reset_index(level=['unit_id', 'dtime'])['lookups']

ut_df = youtube_ttfb_year_median.reset_index(level=['unit_id', 'dtime'])
ut_df['fetch_time'] = youtube_ft_year_median.reset_index(level=['unit_id', 'dtime'])['fetch_time']
ut_df['lookup_avg_time'] = youtube_dnst_year_median.reset_index(level=['unit_id', 'dtime'])['lookup_avg_time']
ut_df['objects'] = youtube_objects_year_median.reset_index(level=['unit_id', 'dtime'])['objects']
ut_df['bytes_total'] = youtube_bytes_year_median.reset_index(level=['unit_id', 'dtime'])['bytes_total']
ut_df['lookups'] = youtube_lookups_year_median.reset_index(level=['unit_id', 'dtime'])['lookups']

In [12]:
print("Median DNS Lookup time (MIN | MED | MAX)")
print("\tFacebook")
for gr, val in fb_df.groupby(['dtime'])['lookup_avg_time']:
    print('\t \t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tGoogle")
for gr, val in ggl_df.groupby(['dtime'])['lookup_avg_time']:
    print('\t\t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tYouTube")
for gr, val in ut_df.groupby(['dtime'])['lookup_avg_time']:
    print('\t\t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))

Median DNS Lookup time (MIN | MED | MAX)
	Facebook
	 	 2014  =  1.41 | 42.12 | 209.8
	 	 2015  =  1.26 | 19.7 | 209.41
	 	 2016  =  1.19 | 17.28 | 265.64
	 	 2017  =  1.1 | 15.83 | 99.89
	Google
		 2014  =  1.61 | 40.56 | 211.38
		 2015  =  1.22 | 18.77 | 213.17
		 2016  =  1.11 | 17.79 | 334.8
		 2017  =  1.0 | 17.8 | 275.78
	YouTube
		 2014  =  1.71 | 45.41 | 212.62
		 2015  =  1.33 | 22.08 | 215.43
		 2016  =  1.2 | 17.66 | 366.82
		 2017  =  0.83 | 12.81 | 287.55


In [13]:
print('changes from 2014 to 2016')
print("Facebook = {}".format(round( (42.12 - 17.28)*100/ 42.12 ), 2))
print("Google = {}".format(round( (40.56 - 17.79)*100/40.56 ), 2))
print("YouTube = {}".format(round( (45.41 - 17.66)*100/45.41 ), 2))

print('\nchanges from 2015 to 2017')
print("Facebook = {}".format(round((19.7 - 15.83)*100/ 19.7 ), 2))
print("Google = {}".format( round((18.77 - 17.8)*100/18.77 ), 2))
print("YouTube = {}".format( round((22.08 - 12.81)*100/22.08 ), 2))

changes from 2014 to 2016
Facebook = 59
Google = 56
YouTube = 61

changes from 2015 to 2017
Facebook = 20
Google = 5
YouTube = 42


In [14]:
print("Median Number of servers (MIN | MED | MAX)")
print("\tFacebook")
for gr, val in fb_df.groupby(['dtime'])['lookups']:
    print('\t \t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tGoogle")
for gr, val in ggl_df.groupby(['dtime'])['lookups']:
    print('\t\t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tYouTube")
for gr, val in ut_df.groupby(['dtime'])['lookups']:
    print('\t\t', gr, ' = ', round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))

Median Number of servers (MIN | MED | MAX)
	Facebook
	 	 2014  =  1 | 1.0 | 2
	 	 2015  =  1 | 1.0 | 1
	 	 2016  =  1 | 1.0 | 2
	 	 2017  =  1 | 1.0 | 2
	Google
		 2014  =  2.0 | 2.0 | 2.0
		 2015  =  1.0 | 2.0 | 2.0
		 2016  =  2.0 | 2.0 | 4.0
		 2017  =  3.0 | 4.0 | 4.0
	YouTube
		 2014  =  3 | 3.0 | 3
		 2015  =  1 | 3.0 | 3
		 2016  =  2 | 2.0 | 2
		 2017  =  1 | 1.0 | 2


In [15]:
print("Median TTFB (MIN | MED | MAX)")
print("\tFacebook")
for gr, val in fb_df.groupby(['dtime'])['ttfb_avg']:
    print('\t \t', gr, ' = ',  round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tGoogle")
for gr, val in ggl_df.groupby(['dtime'])['ttfb_avg']:
    print('\t\t', gr, ' = ',  round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))
    
print("\tYouTube")
for gr, val in ut_df.groupby(['dtime'])['ttfb_avg']:
    print('\t\t', gr, ' = ',  round(val.min(), 2), '|', round(val.median(), 2) , '|', round(val.max(), 2))

Median TTFB (MIN | MED | MAX)
	Facebook
	 	 2014  =  84.86 | 170.55 | 503.2
	 	 2015  =  60.59 | 153.08 | 568.04
	 	 2016  =  0.0 | 146.66 | 1361.89
	 	 2017  =  46.18 | 173.05 | 1507.07
	Google
		 2014  =  32.2 | 114.51 | 825.87
		 2015  =  0.0 | 80.15 | 626.05
		 2016  =  17.0 | 71.49 | 524.11
		 2017  =  15.24 | 57.61 | 1167.53
	YouTube
		 2014  =  19.57 | 100.24 | 790.66
		 2015  =  0.0 | 128.38 | 505.87
		 2016  =  36.86 | 118.84 | 544.5
		 2017  =  40.47 | 120.84 | 1447.38


In [16]:
print("Median PLT")
print("\tFacebook")
for gr, val in fb_df.groupby(['dtime'])['fetch_time']:
    print('\t \t', gr, ' = ', round(val.median(), 2))
    
print("\tGoogle")
for gr, val in ggl_df.groupby(['dtime'])['fetch_time']:
    print('\t\t', gr, ' = ', round(val.median(), 2))
    
print("\tYouTube")
for gr, val in ut_df.groupby(['dtime'])['fetch_time']:
    print('\t\t', gr, ' = ', round(val.median(), 2))

Median PLT
	Facebook
	 	 2014  =  290.87
	 	 2015  =  490.89
	 	 2016  =  520.22
	 	 2017  =  595.37
	Google
		 2014  =  1428.27
		 2015  =  686.98
		 2016  =  500.98
		 2017  =  408.14
	YouTube
		 2014  =  1446.87
		 2015  =  1206.72
		 2016  =  637.5
		 2017  =  641.4
